In [5]:
import pandas as pd
import numpy as np
import os
from geopy.geocoders import Nominatim
from contextlib import suppress
import math

In [ ]:
crops=['Bajra','Rice']
x='wheat'

In [6]:
#shorten (manually update big_states and path for each crop)
path=r'C:\Users\ABC\Desktop\BE Project\from milan\bajra'
big_states=['Maharashtra','Rajasthan']
big_state_shortened_length=[]
big_state_length=[]

for big_state in big_states:
    print(big_state)
    list_df=[]
    for j in range(1,20):
        try:
            print(j)
            df=pd.read_csv(os.path.join(path,'Bajra_'+big_state+str(j)+'.csv'))
            list_df.append(df)
            
        except:
            break   
    df1=pd.concat(list_df)
    df2=df1.iloc[::math.ceil(len(df1)/100000)]
    big_state_length.append(len(df1))
    big_state_shortened_length.append(len(df2))
    
    df2.to_csv(os.path.join(path,'Rice_'+big_state+'.csv'))

Maharashtra
1
2
3
4
Rajasthan
1
2
3
4


In [2]:
#iterate thru crop folders
    #iterate thru 'crop_state.csv' in that folder
    #preprocess and combine all states of a single crop and save into the same folder

In [22]:
files=os.listdir(path)
for file in files:
    df=pd.read_csv(os.path.join(path,file))
    
    if 'Unnamed: 0' in df.columns:
        del df['Unnamed: 0']
        
    del df['Sl no.']
    del df['Variety']
    del df['Grade']
    del df['District Name']
    df['Latitude']=[0.00]*len(df)
    df['Longitude']=[0.00]*len(df)

    markets=df['Market Name'].unique()

    lat={}
    long={}

    for market in markets:
        try:
            address=market.split('(')[0]
            geolocator = Nominatim(user_agent="Your_Name")
            location = geolocator.geocode(address)
            lat[market]=location.latitude
            long[market]=location.longitude
        except:
            lat[market]=0.00
            long[market]=0.00

    for i in range(len(df)):
        df.Latitude[i]=float(lat[df['Market Name'][i]])
        df.Longitude[i]=float(long[df['Market Name'][i]])

    df['Date']=[int(date[0:2]) for date in df['Price Date']]
    df['Month']=[date[3:6] for date in df['Price Date']]
    df['Year']=[int(date[7:9]) for date in df['Price Date']]
    del df['Market Name']
    del df['Price Date']

    months={'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}
    df['Month']=[months[month] for month in df['Month']]
    df=df.rename(columns={"Min Price (Rs./Quintal)":"Min Price","Max Price (Rs./Quintal)":"Max Price","Modal Price (Rs./Quintal)":'Modal Price'})

    df=df[df['Latitude']!=0]

    path='C:/Users/ABC/Desktop/BE Project/from milan/bajra'
    df.to_csv(os.path.join(path,file[:file.index('.')]+'_preproc.csv'))

c:\users\abc\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\abc\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [31]:
#combine .csvs for each crop and save in /crop folder
#remove rows with 0 as Min Price
#in each crop folder, combine all crop_state.csv files and save as crop_final.csv

In [25]:
list_df=[]
files=os.listdir('bajra/')

for file in files:
    df=pd.read_csv('bajra/'+file)
    list_df.append(df)

final_df=pd.concat(list_df).drop(['Unnamed: 0'],axis=1)
final_data=final_df[(final_df['Min Price']!=0) & (final_df['Max Price']!=0) & (final_df['Modal Price']!=0)]
final_data

,Commodity,Min Price,Max Price,Modal Price,Latitude,Longitude,Date,Month,Year
0,Bajra(Pearl Millet/Cumbu),1885.0,1945.0,1915.0,28.613178,76.998285,16,12,19
1,Bajra(Pearl Millet/Cumbu),1825.0,1995.0,1910.0,28.613178,76.998285,15,12,19
2,Bajra(Pearl Millet/Cumbu),1950.0,1950.0,1950.0,28.613178,76.998285,14,12,19
3,Bajra(Pearl Millet/Cumbu),1841.0,1850.0,1845.0,28.613178,76.998285,12,12,19
4,Bajra(Pearl Millet/Cumbu),1870.0,2040.0,1955.0,28.613178,76.998285,10,12,19
...,...,...,...,...,...,...,...,...,...
48937,Bajra(Pearl Millet/Cumbu),1900.0,2000.0,1950.0,28.785205,77.586326,18,12,18
48938,Bajra(Pearl Millet/Cumbu),1900.0,2000.0,1950.0,28.785205,77.586326,17,12,18
48939,Bajra(Pearl Millet/Cumbu),750.0,850.0,750.0,28.785205,77.586326,29,11,11
48940,Bajra(Pearl Millet/Cumbu),800.0,850.0,800.0,28.785205,77.586326,26,11,11


In [26]:
final_data.to_csv('bajra/bajra_final.csv')